the data we are interested in is VALENCE.PLEASANTNESS, wich take values between 0 and 100. The objective is to create a model estimating the VALENCE.PLEASANTNESS given the inputs "Intensity, "SWEETORSOUR"(sure?), and V2 to V11787, which represent the presence or not of a corresponding physiochemical feature in the smelled molecule. At first sight of the data plotted above, we can see that the SWEETORSOUR variable seems have a relatively big impact on VALENCE.PLEASANTNESS (actually it does not proves anything, it is just an early observation). Now we look at some randomly chosen physiochemical features comparing the means of VALENCE.PLEASANTNESS when they are present or not.:

In [3]:
#check the means with or without some randomly chosen physiochemical features
for(col in sample(2:nrow(data),7)){
    idx.presence<-c()
    idx.nonpresence<-c()
    for(i in 1:nrow(data)){
        if(data[col,i]==1){
            idx.presence<-append(idx.presence,i)
        }else{
            idx.nonpresence<-append(idx.nonpresence,i)
        }
    }
    #message("presence mean of ", colnames(data)[col]," ",mean(data$VALENCE.PLEASANTNESS[idx.presence]))
    #message("nonpresence mean ",colnames(data)[col]," ",mean(data$VALENCE.PLEASANTNESS[idx.nonpresence]))
}

Now, Let's start to create a basic linear regression model on all the data. This model will be the base to be improved to try and get the best linear model. We separate all the data in a training set (2/3 of the whole data is taken as training set because we will separate it in a validation and actual training set later).

In [31]:
#create a training (for now, it will be split later) and test set.
idx.train<-sample(nrow(data),nrow(data)*(2/3))
data.test<-data[-idx.train,]
data<-data[idx.train,]

the next cell is really slow, because of all the predictors (11789 !!!), and since p>>n we will largely overfit the data.

In [6]:
basic.lm.fit<-lm(VALENCE.PLEASANTNESS~.,data=data)

In [14]:
train.pred<-predict(basic.lm.fit,data=data)
test.pred<-predict(basic.lm.fit,data=data.test)
trainMSE<-mean((data$VALENCE.PLEASANTNESS-train.pred)^2)
testMSE<-mean((data.test$VALENCE.PLEASANTNESS-test.pred)^2)
cat("train MSE: ",trainMSE, "\n")
cat("test MSE: ",testMSE)

#summary(basic.lm.fit)    


ERROR: Error in predict(basic.lm.fit, data = data): objet 'basic.lm.fit' introuvable


We can see that, as predicted, we largely overfit the data when using all predictors for linear regression. Also, consequently to p>>n, a large part of the predictors parameters are NAs. We will use cross validation to try and get the number of predictors that gives the lowest validation error.

In [15]:
library(tidymodels)
library(leaps)

In [32]:
#testing cell
validation.data<-vfold_cv(data, v=5)
fit<-regsubsets(VALENCE.PLEASANTNESS~.,data, method="forward", nvmax=20)

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"11487  linear dependencies found"


Reordering variables and trying again:


ERROR: Error in leaps.setup(x[, ii[reorder], drop = FALSE], y, wt, force.in[reorder], : NA/NaN/Inf dans un appel à une fonction externe (argument 4)


In [8]:
validation.data<-vfold_cv(data, v=5)

cv_fit_and_eval <- function(fold, formula=VALENCE.PLEASANTNESS ~ .) {
    fit<-regsubsets(formula, analysis(fold), method="forward", nvmax=30)
    validation.set<-assessment(fold)
    sapply(seq(1,fit$nvmax-1),
           function(idx) fitmean(validation.set$VALENCE.PLEASANTNESS-predict(fit,validation.set,idx,formula)^2))
    str(fit)
}
cv.errors<-sapply(validation.data$splits,cv_fit_and_eval)

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"11519  linear dependencies found"


Reordering variables and trying again:


ERROR: Error in leaps.setup(x[, ii[reorder], drop = FALSE], y, wt, force.in[reorder], : NA/NaN/Inf dans un appel à une fonction externe (argument 4)


In [70]:
fits<-regsubsets(VALENCE.PLEASANTNESS~., data, method = "forward", nvmax=30)

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2700  linear dependencies found"


Reordering variables and trying again:


In [76]:
predict.regsubsets <- function(object, newdata, id, form = as.formula(object$call[[2]])) {
    mat = model.matrix(form, newdata)
    coefi = coef(object, id=id)
    xvars = names(coefi)
    mat[,xvars]%*%coefi
}                                    #using predict formula defined in Lecture 4

In [78]:
fit.and.eval<-function(fold,formula = VALENCE.PLEASANTNESS ~ .) {
    fit<-regsubsets(formula, analysis(fold), nvmax=30, method = 'forward')
    valid.set<-assessment(fold)
    sapply(1:(fit$nvmax-1),
          function(nb) mean((valid.set$VALENCE.PLEASANTNESS - predict(fit, valid.set, nb, formula))^2))
}
validation.data<-vfold_cv(data,v=5)
cv.errors<-sapply(validation.data$splits, fit.and.eval)
head(cv.errors)


Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2710  linear dependencies found"


Reordering variables and trying again:


Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2711  linear dependencies found"


Reordering variables and trying again:


Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2708  linear dependencies found"


...

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2712  linear dependencies found"


Reordering variables and trying again:


Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"2709  linear dependencies found"


Reordering variables and trying again:


253.2524,350.4810,295.1347,349.3675,329.1570
256.7313,358.4596,280.5217,351.1872,322.3044
256.5084,367.3672,319.5868,349.1995,324.1539
262.6435,362.4751,326.9709,355.6666,325.4438
263.9150,358.9280,318.6749,344.6454,318.3929
257.9440,357.2943,307.0730,335.7402,307.3385


In [82]:
head(cv.errors,10)
which.min(rowMeans(cv.errors))

253.2524,350.4810,295.1347,349.3675,329.1570
256.7313,358.4596,280.5217,351.1872,322.3044
256.5084,367.3672,319.5868,349.1995,324.1539
262.6435,362.4751,326.9709,355.6666,325.4438
263.9150,358.9280,318.6749,344.6454,318.3929
257.9440,357.2943,307.0730,335.7402,307.3385
255.4004,356.0790,314.7202,336.8180,302.0043
265.9355,363.2667,311.2241,331.1116,301.1186
263.6568,360.2667,316.6011,333.9795,299.3935
266.3011,360.9370,323.9093,336.1302,303.3486


[1] 7

In [76]:
pred.to.rm<-apply(max.cor.pred,1,function(x) x[1])
length(pred.to.rm)
pred.to.rm<-pred.to.rm[!duplicated(pred.to.rm)]
typeof(pred.to.rm)
pred.to.rm


[1] 1931

[1] "character"

[1] "87"      "526"     "699"     "1822"    "2072"    "2625"    "3078"   
  [8] "8773"    "8793"    "10731"   "10878"   "11026"   "11175"   "11316"  
 [15] "11386"   "12402"   "12712"   "16081"   "21003"   "24753"   "26334"  
 [22] "29146"   "30606"   "38776"   "39055"   "40487"   "41316"   "41712"  
 [29] "43951"   "44348"   "44848"   "45148"   "45751"   "46054"   "46663"  
 [36] "46961"   "47585"   "47586"   "47895"   "48205"   "49427"   "50058"  
 [43] "51675"   "52320"   "57613"   "57917"   "58294"   "59336"   "62830"  
 [50] "63185"   "64413"   "67140"   "67502"   "67870"   "71592"   "71622"  
 [57] "73148"   "73531"   "73919"   "74465"   "74681"   "75466"   "75611"  
 [64] "76245"   "77313"   "77751"   "77788"   "78183"   "78589"   "82210"  
 [71] "83434"   "83843"   "84664"   "85076"   "85903"   "86310"   "87152"  
 [78] "87153"   "87571"   "87990"   "89648"   "90497"   "95695"   "97015"  
 [85] "97456"   "98341"   "99681"   "100128"  "100576"  "102350"  "103257" 
 [92] "107411"  "107880"  "109276"  "109744"  "110683"  "111154"  "112099" 
 [99] "114002"  "114003"  "114481"  "114960"  "116856"  "117825"  "118311" 
[106] "118798"  "119302"  "119775"  "121741"  "122260"  "122760"  "123232" 
[113] "123723"  "124249"  "124748"  "125220"  "125749"  "126251"  "126726" 
[120] "127258"  "127735"  "128245"  "129285"  "129286"  "129795"  "130305" 
[127] "130786"  "131298"  "131811"  "132325"  "132840"  "133356"  "138070" 
[134] "138601"  "140183"  "140713"  "141776"  "142309"  "143378"  "144441" 
[141] "145529"  "146069"  "146070"  "146611"  "148755"  "149848"  "150365" 
[148] "154841"  "155403"  "156459"  "157078"  "157639"  "158764"  "159328" 
[155] "160459"  "161584"  "162734"  "163305"  "163306"  "163878"  "165339" 
[162] "165539"  "166146"  "166692"  "167301"  "172573"  "173166"  "174934" 
[169] "175526"  "176713"  "177308"  "178501"  "180887"  "180900"  "181502" 
[176] "182105"  "184498"  "185715"  "186294"  "187533"  "191266"  "191890" 
[183] "193070"  "193751"  "194374"  "194939"  "195623"  "196249"  "196817" 
[190] "197504"  "198753"  "200027"  "200601"  "200660"  "200661"  "201295" 
[197] "203142"  "203811"  "204419"  "205090"  "223417"  "223842"  "224756" 
[204] "228422"  "228797"  "232192"  "232211"  "236320"  "239084"  "243922" 
[211] "244621"  "245332"  "249542"  "255226"  "255941"  "260279"  "264569" 
[218] "265327"  "266056"  "266786"  "269686"  "271187"  "274901"  "277856" 
[225] "287632"  "288391"  "289162"  "289882"  "290644"  "292171"  "293732" 
[232] "294469"  "295237"  "296006"  "297547"  "299092"  "299866"  "300671" 
[239] "301387"  "301417"  "302194"  "304531"  "305369"  "309232"  "310019" 
[246] "310837"  "311626"  "312416"  "318927"  "319600"  "334942"  "335761" 
[253] "336581"  "412679"  "660713"  "662972"  "666432"  "667587"  "671052" 
[260] "672217"  "673373"  "674534"  "675696"  "676849"  "680353"  "682688" 
[267] "755806"  "768171"  "771480"  "783117"  "784369"  "785630"  "786884" 
[274] "788139"  "789395"  "790652"  "791910"  "792072"  "792244"  "802010" 
[281] "804545"  "805814"  "807084"  "808355"  "809627"  "817276"  "817280" 
[288] "818559"  "819839"  "821120"  "822402"  "823685"  "824969"  "826254" 
[295] "827540"  "827712"  "837864"  "839159"  "840455"  "841752"  "843050" 
[302] "844349"  "845649"  "853470"  "854777"  "856085"  "857394"  "858704" 
[309] "860015"  "861327"  "862640"  "863954"  "874502"  "877149"  "878474" 
[316] "879800"  "881127"  "882455"  "885059"  "890445"  "891780"  "893116" 
[323] "894453"  "895791"  "897130"  "898470"  "899811"  "901153"  "910575" 
[330] "911772"  "932132"  "952717"  "959771"  "961157"  "962544"  "963932" 
[337] "965321"  "966711"  "968102"  "969494"  "970887"  "972281"  "973676" 
[344] "975072"  "976469"  "977867"  "979266"  "980666"  "982067"  "983469" 
[351] "984872"  "986276"  "987681"  "989087"  "990494"  "991902"  "993311" 
[358] "994721"  "996132"  "997544"  "998957"  "1000371" "1001786" "1003202"
[365] "1004619" "1006037" "1007456" "1022947" "104